## Question 4 & 5


4. Visit python.langchain.com/docs/modules/model_io/prompts/example_selectors/ngram_overlap/ and implement n-gram overlap selector

5. Visit python.langchain.com/docs/modules/model_io/prompts/example_selectors/similarity and implement semantic similarity selector

## **Import Files**

In [1]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.example_selectors import NGramOverlapExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import AzureOpenAIEmbeddings


In [2]:

# initializing the embeddin model
embeddings = AzureOpenAIEmbeddings(
    model="ADA_RAG_DONO_DEMO",
    api_key="35e177acfb054c28a1058071beb6e609",
    api_version="2024-02-01",
    azure_endpoint="https://dono-rag-demo-resource-instance.openai.azure.com"
)

## Semantic Similarity Selector

In [3]:
#creating the examples
example_prompt = PromptTemplate(
    input_variables=["sentence", "subject"],
    template="Sentence: {sentence}\nSubject: {subject}",
)

examples = [
    {"sentence": "The cat sat on the mat.", "subject": "The cat"},
    {"sentence": "She enjoys reading books.", "subject": "She"},
    {"sentence": "The children are playing in the park.", "subject": "The children"},
    {"sentence": "John and Mary went to the store.", "subject": "John and Mary"},
    {"sentence": "A dog barked loudly at the stranger.", "subject": "A dog"},
    {"sentence": "The teacher explained the lesson.", "subject": "The teacher"},
    {"sentence": "Birds are singing in the trees.", "subject": "Birds"},
    {"sentence": "My brother loves to play basketball.", "subject": "My brother"},
    {"sentence": "The book was on the table.", "subject": "The book"},
    {"sentence": "Cars zoomed past the busy intersection.", "subject": "Cars"},
    {"sentence": "Her smile brightened the room.", "subject": "Her smile"},
    {"sentence": "They will arrive tomorrow.", "subject": "They"},
]


In [4]:
# defining the example selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=2,
)

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the subject of every input sentence",
    suffix="Input: {sentence}\nOutput:",
    input_variables=["sentence"],
)

In [5]:
# Result
print(similar_prompt.format(sentence="The quick brown fox jumps over the lazy dog."))

Give the subject of every input sentence

Sentence: The cat sat on the mat.
Subject: The cat

Sentence: A dog barked loudly at the stranger.
Subject: A dog

Input: The quick brown fox jumps over the lazy dog.
Output:


In [6]:
# Result
print(similar_prompt.format(sentence="Alice and Bob are going to the concert."))

Give the subject of every input sentence

Sentence: John and Mary went to the store.
Subject: John and Mary

Sentence: They will arrive tomorrow.
Subject: They

Input: Alice and Bob are going to the concert.
Output:


In [7]:
# Checking after adding new examples
similar_prompt.example_selector.add_example(
    {"sentence": "The teacher explained the lesson.", "subject": "The teacher"}
)
similar_prompt.example_selector.add_example(
    {"sentence": "Her smile brightened the room.", "subject": "Her smile"}
)
print(similar_prompt.format(sentence="Our teacher teaches good."))

Give the subject of every input sentence

Sentence: The teacher explained the lesson.
Subject: The teacher

Sentence: The teacher explained the lesson.
Subject: The teacher

Input: Our teacher teaches good.
Output:


## N-Gram Overlap Selector

In [8]:
# creating the n-gram example selector
example_selector = NGramOverlapExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    threshold=-1.0,
)

dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the subject of every input sentence",
    suffix="Input: {sentence}\nOutput:",
    input_variables=["sentence"],
)

In [9]:
# result
print(dynamic_prompt.format(sentence="The quick brown fox jumps over the lazy dog."))

Give the subject of every input sentence

Sentence: The cat sat on the mat.
Subject: The cat

Sentence: The children are playing in the park.
Subject: The children

Sentence: The teacher explained the lesson.
Subject: The teacher

Sentence: The book was on the table.
Subject: The book

Sentence: John and Mary went to the store.
Subject: John and Mary

Sentence: A dog barked loudly at the stranger.
Subject: A dog

Sentence: Birds are singing in the trees.
Subject: Birds

Sentence: Cars zoomed past the busy intersection.
Subject: Cars

Sentence: Her smile brightened the room.
Subject: Her smile

Sentence: She enjoys reading books.
Subject: She

Sentence: My brother loves to play basketball.
Subject: My brother

Sentence: They will arrive tomorrow.
Subject: They

Input: The quick brown fox jumps over the lazy dog.
Output:


In [10]:
# checking after changing the threshold
example_selector.threshold = 0.028
print(dynamic_prompt.format(sentence="The quick brown fox jumps over the lazy dog."))

Give the subject of every input sentence

Sentence: The cat sat on the mat.
Subject: The cat

Sentence: The children are playing in the park.
Subject: The children

Sentence: The teacher explained the lesson.
Subject: The teacher

Sentence: The book was on the table.
Subject: The book

Input: The quick brown fox jumps over the lazy dog.
Output:
